In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

Mounted at /content/drive


In [ ]:
# data = pd.read_csv('/content/drive/MyDrive/Gaucho/Copy of nfl_playoff_data.csv', low_memory=False)
data = pd.read_csv('/content/drive/MyDrive/UCSB/GSA/Python Files/SuperBowl/nfl_playoff_data.csv', low_memory=False)

# Remove remaining rows with missing values
data = data.dropna()

# Print missing value counts (there shouldn't be any left at this point)
print("\nMissing Values:")
missing_values = data.isnull().sum().sort_values(ascending=False)
print(missing_values[missing_values > 0])

# Drop columns we don't plan on using
data = data.drop(columns=['game_type', 'week', 'season', 'opponent', 'roof', 'surface', 'team', 'referee'])

# Print a list of remaining columns
data.columns

data.head()


Missing Values:
Series([], dtype: int64)


,points_favored,over_odds,under_odds,temp,wind,win,team_win_pct,team_net_points,team_sov,team_sos,opponent_win_pct,opponent_net_points,opponent_sov,opponent_sos
45,1.0,-104.0,-106.0,44.0,15.0,True,0.5625,-6,0.381944,0.453125,0.5625,75,0.437500,0.457031
46,9.5,-108.0,-102.0,50.0,8.0,True,0.7500,148,0.468750,0.496094,0.6250,21,0.400000,0.468750
47,6.5,-107.0,-103.0,52.0,5.0,True,0.6250,70,0.450000,0.476562,0.5000,-7,0.421875,0.519531
48,4.0,-118.0,108.0,63.0,4.0,False,0.8125,152,0.447115,0.460938,0.7500,67,0.505208,0.500000
51,5.0,-102.0,-108.0,53.0,6.0,False,0.8750,189,0.424107,0.457031,0.7500,148,0.468750,0.496094


In [ ]:
features = ['points_favored', 'temp', 'team_win_pct', 'team_net_points', 'opponent_win_pct', 'opponent_net_points']
target = 'win'

X = data[features]
y = data[target]

from sklearn.model_selection import train_test_split


#this splits our data into X_train and y_train (our training data) and X_test and y_test (our testing data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)#mess around with the test_size if you want, don't worry about the random_state

y_train = y_train.astype(int)  # Convert to integer type
y_test = y_test.astype(int)

from sklearn.metrics import accuracy_score, classification_report

def print_model_results(model, y_pred):
  print(f"Total 0s and 1s predicted: {np.unique(y_pred, return_counts=True)}")
  print(f"Total 0s and 1s in test set: {np.unique(y_test, return_counts=True)}\n")

  accuracy = accuracy_score(y_test, y_pred)
  print(f"Accuracy: {accuracy:.2f}\n")

  print(f"Classification Report: \n{classification_report(y_test, y_pred)}")

  proba = model.predict_proba(X_test)
  print(f"Probability of 0 or 1, respectively, for each play: \n{proba}")

import xgboost as xgb
xg_model = xgb.XGBClassifier()

xg_model.fit(X_train, y_train)
y_pred = xg_model.predict(X_test)

print_model_results(xg_model, y_pred=y_pred)

sb_2025 = data.tail(1)
data = data.drop(data.tail(1).index)
sb_2025
# 0 = KC, 1 = PHI
sb_2025_filtered = sb_2025.filter(features, axis = 1)
sb_2025_prediction = xg_model.predict(sb_2025_filtered)
print(sb_2025_prediction) #Philadelphia will win with accuracy

Total 0s and 1s predicted: (array([0, 1]), array([11, 19]))
Total 0s and 1s in test set: (array([0, 1]), array([11, 19]))

Accuracy: 0.73

Classification Report: 
              precision    recall  f1-score   support

           0       0.64      0.64      0.64        11
           1       0.79      0.79      0.79        19

    accuracy                           0.73        30
   macro avg       0.71      0.71      0.71        30
weighted avg       0.73      0.73      0.73        30

Probability of 0 or 1, respectively, for each play: 
[[0.06828523 0.9317148 ]
 [0.91565835 0.08434162]
 [0.03286654 0.96713346]
 [0.03814191 0.9618581 ]
 [0.07137257 0.92862743]
 [0.01096803 0.989032  ]
 [0.04523426 0.95476574]
 [0.55074567 0.44925433]
 [0.04502177 0.9549782 ]
 [0.9695576  0.03044243]
 [0.17031395 0.82968605]
 [0.04299897 0.95700103]
 [0.33230197 0.667698  ]
 [0.8000956  0.1999044 ]
 [0.08009046 0.91990954]
 [0.9700125  0.0299875 ]
 [0.52908546 0.47091454]
 [0.9740512  0.0259488 ]
 [0.129